![alt text](https://zewailcity.edu.eg/main/images/logo3.png)

_Prepared by_  [**Muhammad Hamdy AlAref**](mailto:malaref@zewailcity.edu.eg)

# Uninformed Search

Uninformed search algorithms try to solve the problem using only its formulation without any specific knowledge about it. While they are general and, theoretically, can solve any solvable problem, they can be very inefficient!

## Problem Formulation

Problem formulation is the first step in solving it! Basically, problem formulation is rephrasing the problem in a structure that can serve as input to a search algorithms. The following class gives a base for such structure!

In [1]:
class Problem:
    '''
    Abstract base class for problem formulation.
    It declares the expected methods to be used by a search algorithm.
    All the methods declared are just placeholders that throw errors if not overriden by child "concrete" classes!
    '''
    
    def __init__(self):
        '''Constructor that initializes the problem. Typically used to setup the initial state and, if applicable, the goal state.'''
        self.init_state = None
    
    def actions(self, state):
        '''Returns an iterable with the applicable actions to the given state.'''
        raise NotImplementedError
    
    def result(self, state, action):
        '''Returns the resulting state from applying the given action to the given state.'''
        raise NotImplementedError
    
    def goal_test(self, state):
        '''Returns whether or not the given state is a goal state.'''
        raise NotImplementedError
    
    def step_cost(self, state, action):
        '''Returns the step cost of applying the given action to the given state.'''
        raise NotImplementedError

## Node Data Structure

Search algorithms construct trees or graphs of atomic states to try to reach their goal. To keep track of that, some bookkeeping is required. The following class encapsulates such bookkeeping capabilities!

In [2]:
class Node:
    '''Node data structure for search space bookkeeping.'''
    
    def __init__(self, state, parent, action, path_cost):
        '''Constructor for the node state with the required parameters.'''
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost

    @classmethod
    def root(cls, init_state):
        '''Factory method to create the root node.'''
        return cls(init_state, None, None, 0)

    @classmethod
    def child(cls, problem, parent, action):
        '''Factory method to create a child node.'''
        return cls(
            problem.result(parent.state, action), #The resulting state
            parent,
            action,
            parent.path_cost + problem.step_cost(parent.state, action))

def solution(node):
    '''A method to extract the sequence of actions representing the solution from the goal node.'''
    actions = []
    cost = node.path_cost
    while node.parent is not None:
        actions.append(node.action)
        node = node.parent
    actions.reverse()
    return actions, cost

## Example Algorithm: Breadth-First Search (BFS)

Breadth-first search is an uninformed algorithm that searches the state space level-by-level starting from the initial node, i.e., it starts by looking at the root node, then its children, its children's children after that, and so on! Here are example implementations for both its tree version and its graph version.

In [3]:
from collections import deque

def bfs_tree(problem, verbose=False):
    '''Breadth-first tree search implementation.'''
    if problem.goal_test(problem.init_state): return solution(problem.init_state)
    frontier = deque([Node.root(problem.init_state)])
    if verbose: visualizer = Visualizer(problem)
    while frontier:
        if verbose: visualizer.visualize(frontier)
        node = frontier.pop()
        for action in problem.actions(node.state):
            child = Node.child(problem, node, action)
            if problem.goal_test(child.state):
                return solution(child)
            frontier.appendleft(child)

def bfs_graph(problem, verbose=False):
    '''Breadth-first graph search implementation.'''
    if problem.goal_test(problem.init_state): return solution(problem.init_state)
    frontier = deque([Node.root(problem.init_state)])
    explored = {problem.init_state}
    if verbose: visualizer = Visualizer(problem)
    while frontier:
        if verbose: visualizer.visualize(frontier)
        node = frontier.pop()
        for action in problem.actions(node.state):
            child = Node.child(problem, node, action)
            if child.state not in explored:
                if problem.goal_test(child.state):
                    return solution(child)
                frontier.appendleft(child)
                explored.add(child.state)

## Visualizer

For those of you with sharp eyes, you may have notice a couple of differences from the book pseudo-code. A very obvious one it the `verbose` flag that enables the `Visualizer`! The `Visualizer` is just an auxiliary class that encapsulates some functionalities that can visualize the algorithm's progress. It greatly slows the algorithm down and it is only for illustration purposes! Here is its code, but feel free to skip it altogether!

In [4]:
from shutil import get_terminal_size
terminal_width, _ = get_terminal_size()

_visualizers = {}

def _default_visualizer(_, state):
    '''Generic visualizer for unknown problems.'''
    print(state)

class Visualizer:
    '''Visualization and printing functionality encapsulation.'''

    def __init__(self, problem):
        '''Constructor with the problem to visualize.'''
        self.problem = problem
        self.counter = 0
    
    def visualize(self, frontier):
        '''Visualizes the frontier at every step.'''
        self.counter += 1
        print(f'Frontier at step {self.counter}')
        for node in frontier:
            print()
            _visualizers.get(type(self.problem), _default_visualizer)(self.problem, node.state)
        print('-' * terminal_width)

## Example: Sliding Puzzle

The [sliding puzzle](https://en.wikipedia.org/wiki/Sliding_puzzle) is a classic toy problem for search algorithms. It consists of a 2D tile board with one blank tile that the neighboring tiles can slide into to give new configurations of the board. Usually, initial and goal configurations are given and the search algorithm should figure out the steps to go from the initial to the goal.

In [5]:
class SlidingPuzzle3x3(Problem):
    '''3x3 Sliding Puzzle problem formulation.'''

    def __init__(self, init_state, goal_state):
        assert init_state.count(' ') == 1
        assert goal_state.count(' ') == 1
        self.init_state = tuple(init_state)
        self._goal_state = tuple(goal_state)
        self._action_values = {'up': -3, 'down': +3, 'left': -1, 'right': +1}
    
    def actions(self, state):
        index = state.index(' ')
        possible_moves = []
        if index // 3 > 0:
            possible_moves.append('up')
        if index // 3 < 2:
            possible_moves.append('down')
        if index % 3 > 0:
            possible_moves.append('left')
        if index % 3 < 2:
            possible_moves.append('right')
        return possible_moves
    
    def result(self, state, action):
        def swap(t, i, j):
            '''Auxiliary function for swapping two elements in a tuple.'''
            l = list(t)
            l[i], l[j] = l[j], l[i]
            return tuple(l)
        index = state.index(' ')
        return swap(state, index, index + self._action_values[action])
    
    def goal_test(self, state):
        return state == self._goal_state
    
    def step_cost(self, state, action):
        return 1

def _sliding_puzzle_3x3_visualizer(problem, state):
    '''Custom visualizer for the 3x3 sliding puzzle problem.'''
    for i in range(0, 9, 3):
        print(' ' + ' '.join(state[i:i + 3]) + ' ')

_visualizers[SlidingPuzzle3x3] = _sliding_puzzle_3x3_visualizer

Let's try solving the sliding puzzle with BFS!

In [6]:
problem = SlidingPuzzle3x3('12345678 ', '123 56478')

In [7]:
bfs_graph(problem, verbose=False)

(['left', 'left', 'up'], 3)

In [8]:
bfs_tree(problem, verbose=False)

(['left', 'left', 'up'], 3)

In [9]:
another_problem = SlidingPuzzle3x3('12345678 ', '12345687 ')
print(bfs_graph(another_problem))

None


What does that mean?

## Requirement

### Part A

Let's re-solve the sliding puzzle problem with other uninformed search algorithms!

You are required to write Python code that implements the following algorithms,

1. Depth-First Search (DFS)
2. Depth-Limited Search (DLS)
3. Iterative Deepening Search (IDS)

You may think this is a lot, but it is not! If you think about it, by implementing the right one first, the other two can be implemented in terms of it!

After the algorithms are implemented, apply them to the sliding puzzle problem and compare the solutions they find with the one breadth-first search found!

**HINT** For DFS and its variants, recursive implementations are easier and more straightforward!

### Part B

Can you formulate the problem of solving a *2D maze*?

Problem formulation is a *major* part of solving it and it can be *deceivingly* simple! This exercises aims to help you practice it.

Think about the best way to formulate the problem, i.e., proper _state_ and _actions_ representations! Then, implement the `Problem` class interface above.

```python
class Maze(Problem):
    ...
```

Following is a sample for constructing a problem instance (NOT a state representation)!

```python
problem = Maze('''
   #                #
   #                #
          ####      #
  ##      #     #####
  *#      #      +   
''')
```

where `*` denotes the starting position (initial state), `+` denotes goals and `#` denotes obstacles. The line's length (number of characters) is the maze width (all the lines are of the same width) and the number of lines is the maze hight. Available actions are `up`, `down`, `left` and `right` provided the destination is not an obstacles or out of the maze bounds.

After you are done, you should be able to solve a problem by calling any search algorithm. For example,

```python
bfs_graph(problem)
```


**Estimated time for this exercise is 1 hour!**

## Part A: Solution

### 1- DFS

**Non-Recursive Implementation for DFS Tree and DFS Graph**

In [10]:
def dfs_tree(problem, verbose=False):
    '''Depth-first tree search implementation.'''
    if problem.goal_test(problem.init_state): return solution(Node.root(problem.init_state))
    frontier = deque([Node.root(problem.init_state)])
    if verbose: visualizer = Visualizer(problem)
    while frontier:
        if verbose: visualizer.visualize(frontier)
        node = frontier.popleft()
        for action in problem.actions(node.state):
            child = Node.child(problem, node, action)
            if problem.goal_test(child.state):
                return solution(child)
            frontier.appendleft(child)

def dfs_graph(problem, verbose=False):
    '''Depthth-first graph search implementation.'''
    if problem.goal_test(problem.init_state): return solution(Node.root(problem.init_state))
    frontier = deque([Node.root(problem.init_state)])
    explored = {problem.init_state}
    if verbose: visualizer = Visualizer(problem)
    while frontier:
        if verbose: visualizer.visualize(frontier)
        node = frontier.popleft()
        for action in problem.actions(node.state):
            child = Node.child(problem, node, action)
            if child.state not in explored:
                if problem.goal_test(child.state):
                    return solution(child)
                frontier.appendleft(child)
                explored.add(child.state)

**Alternatively, Recursive Implementation for DFS Tree and DFS Graph**

In [11]:
def rec_dfs_tree(problem, verbose=False):
    '''Depth-first tree search recursive implementation.'''
    def Reverse(lst):
        lst.reverse()
        return lst
    def recursive_procedure(node):
        print("state= {}".format(node.state))
        ''' Base Case'''
        if problem.goal_test(node.state):
            return solution(node)
        ''' Regular Case'''
        for action in Reverse(problem.actions(node.state)):
            child = Node.child(problem, node, action)
            return(recursive_procedure(child))
        
    ''' Initial Base case'''
    if problem.goal_test(problem.init_state): return solution(Node.root(problem.init_state))
    initial_node = Node.root(problem.init_state)
    if verbose: visualizer = Visualizer(problem)
    return(recursive_procedure(initial_node))



def rec_dfs_graph(problem, verbose=False):
    '''Depth-first graph search recursive implementation.'''
    def Reverse(lst):
        # This REVERSE function is only here to have similar steps compared to the non-recursive implementaion
        lst.reverse()
        return lst
    def recursive_procedure(explored, node):
        explored.add(node.state)
        ''' Base Case'''
        if problem.goal_test(node.state):
            return solution(node)
        else:
            ''' Regular Action'''
            for action in Reverse(problem.actions(node.state)):
                child = Node.child(problem, node, action)
                if child.state not in explored:
                    return(recursive_procedure(explored,child))
    ''' Initial Base Case'''
    if problem.goal_test(problem.init_state): return solution(problem.init_state)
    initial_node = Node.root(problem.init_state)
    explored = set()
    # Go recursively
    return(recursive_procedure(explored, initial_node))

In [12]:
rec_dfs_graph(problem, verbose=True) 

(['left', 'left', 'up'], 3)

In [13]:
dfs_graph(problem, verbose=True)

Frontier at step 1

 1 2 3 
 4 5 6 
 7 8   
--------------------------------------------------------------------------------
Frontier at step 2

 1 2 3 
 4 5 6 
 7   8 

 1 2 3 
 4 5   
 7 8 6 
--------------------------------------------------------------------------------
Frontier at step 3

 1 2 3 
 4 5 6 
   7 8 

 1 2 3 
 4   6 
 7 5 8 

 1 2 3 
 4 5   
 7 8 6 
--------------------------------------------------------------------------------


(['left', 'left', 'up'], 3)

In [ ]:
dfs_tree(problem, verbose=True) #infinite loop

In [ ]:
rec_dfs_tree(problem, verbose=True) #infinite loop (max. recursion depth exceeded)

### 2- DLS

In [14]:
def dls_tree(problem, limiting_depth):
    '''Depth-limited tree search implementation.'''
    def Reverse(lst):
        lst.reverse()
        return lst
    def appendleft_list(stack, lst):
        #Append list in
        for item in Reverse(lst):
            stack.appendleft(item)
        return stack
    
    depth, flag = 0, 0
    if problem.goal_test(problem.init_state): return solution(Node.root(problem.init_state))
    frontier = deque([[Node.root(problem.init_state)]])
    while frontier:
        frontier_lst = []
        nodes = frontier.popleft()
        if flag:
            depth -= 1
        for node in nodes:
            for action in problem.actions(node.state):
                child = Node.child(problem, node, action)
                if problem.goal_test(child.state):
                    return solution(child)
                frontier_lst.append(child) #FIFO (list append)
        if depth < limiting_depth: 
            frontier.appendleft(frontier_lst)
            depth += 1
            flag = 0


In [15]:
dls_tree(problem, 3)

(['left', 'left', 'up'], 3)

### 3- IDS

In [16]:
def ids_tree(problem, depth_range):
    for depth in depth_range:
        sol = dls_tree(problem, depth)
        if sol:
            print('At depth= {}, got a solution {}'.format(depth, sol))
            return sol
        else:
             print('At depth= {}, no solution found!'.format(depth))


In [17]:
ids_tree(problem, range(1,10))

At depth= 1, no solution found!
At depth= 2, got a solution (['left', 'left', 'up'], 3)


(['left', 'left', 'up'], 3)

## Part B: Solution

To construct the maze Map, only enter a string whose length is M*N where M,N are the 2D maze dimension. For this specific implementation M=N=8. The string is read a row by row, i.e the first 8 chars represent the first row starting from the left column towards right one, the next 8 chars represent the second row, etc.

In [18]:
import numpy as np
class Maze(Problem):
    '''Maze problem formulation.'''

    def __init__(self, maze_map):
        assert len(maze_map) == 64
        self.init_state = [maze_map.find('*')//8, maze_map.find('*')%8]
        self._goal_state = [maze_map.find('o')//8, maze_map.find('o')%8]
        print('starting_position ={}'.format(self.init_state))
        print('goal_position ={}'.format(self._goal_state))
        self._action_values = {'up': [1,0], 'down': [-1,0], 'left': [0,-1], 'right': [0,1]}
    
    def actions(self, state):
        row, col = state
        possible_moves = []
        if row != 7:
            if maze_map[(row+1)*8 + col] != 'x':
                possible_moves.append('up')
        if row != 0:
            if maze_map[(row-1)*8 + col] != 'x':
                possible_moves.append('down')
        if col != 0:
            if maze_map[(row)*8 + col-1] != 'x':
                possible_moves.append('left')
        if col != 7:
            if maze_map[(row)*8 + col+1] != 'x':
                possible_moves.append('right')
        return possible_moves
    
    def result(self, state, action):
        return list(np.add(state, self._action_values[action]))
    
    def goal_test(self, state):
        return state == self._goal_state
    
    def step_cost(self, state, action):
        return 1

maze_map = '  *      x x      xx      xxo      xx      xx      x      xx    '
maze_problem = Maze(maze_map)

starting_position =[0, 2]
goal_position =[3, 4]


In [19]:
ids_tree(maze_problem, range(1,10))

At depth= 1, no solution found!
At depth= 2, no solution found!
At depth= 3, no solution found!
At depth= 4, got a solution (['right', 'right', 'up', 'up', 'up'], 5)


(['right', 'right', 'up', 'up', 'up'], 5)

In [20]:
bfs_tree(maze_problem)

(['right', 'right', 'up', 'up', 'up'], 5)